### Imports

In [1]:
import numpy as np
import pandas as pd

### Read data from logs
Each of the provided logs contains frequencies for each of the 1280 tero-instances, for 100 pufs (100 different board)

In [2]:
def extract_frequencies(csv_file, counter_bits):
    data = pd.read_csv(csv_file, sep=',', skiprows=1, names=["BoardNumber","DataSent", "#Sent", "DataReceived", "#Received"])
    responsesRaw = np.array(data['DataReceived'])
    responses = []
    for responseRaw in responsesRaw:
        response = []
        for i in range(0, len(responseRaw), int(counter_bits/4)):
            response.append(int(responseRaw[i:i+int(counter_bits/4)], 16))
        responses.append(response)
    responses = np.array(responses)
    return np.transpose(responses) # for each row, an instance of tero. for each column, the value in that fpga

In [3]:
freqs1 = extract_frequencies("TERO/log_20161008U235656_Small_ChosenRepetition_ChosenTime_22.0C.csv", 32)
freqs2 = extract_frequencies("TERO/log_20161009U002207_Small_ChosenRepetition_ChosenTime_22.0C.csv", 32)
freqs3 = extract_frequencies("TERO/log_20161009U023930_Small_ChosenRepetition_ChosenTime_-0.0C.csv", 32)
freqs4 = extract_frequencies("TERO/log_20161009U110717_Small_ChosenRepetition_ChosenTime_44.1C.csv", 32)

In [4]:
print(freqs1.shape)
print(freqs1)

(1280, 100)
[[  71870  239361  165805 ...  126003   96193   72745]
 [  61075 1131202  936723 ...   63300  122902  123455]
 [  68815   93162  373604 ...   43073   73202  447935]
 ...
 [  85216  104602   91431 ...  142617  219258  164340]
 [ 163790   66403  118687 ...  133235  124278  148629]
 [ 143384  118096   52582 ...  194886   83166  447925]]


### Group frequencies in batches

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16

[0] 0 4 8 12 16
[1] 1 5 9 13 17

[0] 0 8 16  x 160
[1] 1 9 17
[2] 2 10 18
[3] 3 ...
[4] 4 12 20
...
[7]

In [6]:
def group_frequencies(freqs):
    # Based on the assumption we have 8 different cell types, we group each type in the right column
    NUM_INSTANCES = freqs.shape[0]  # 1280
    NUM_FPGA = freqs.shape[1]   # 100
    NUM_TYPES = 8
    BATCH_SIZE = 16
    batchArray = np.zeros((int(NUM_INSTANCES/NUM_TYPES),NUM_TYPES,NUM_FPGA))
    for fpga in range(0, NUM_FPGA):
        for instance_type in range(0, NUM_TYPES):
            batchArray[:,instance_type,fpga] = freqs[instance_type::NUM_TYPES,fpga] # from the starting index, we sample every 8 elements
    # batcharray: 160x8x100
    BATCH_COLUMNS = int(NUM_INSTANCES/BATCH_SIZE)   # 80
    batchResponse = np.zeros((BATCH_SIZE,BATCH_COLUMNS,NUM_FPGA))
    for fpga in range(0, NUM_FPGA):
        fpgaBatch = np.zeros((BATCH_SIZE, BATCH_COLUMNS))
        col = 0
        for instance_type in range(0,NUM_TYPES):
            # get old column
            current_col = batchArray[:,instance_type,fpga]
            # create 10 new columns
            for index in range(0, len(current_col), BATCH_SIZE):
                fpgaBatch[:,col] = current_col[index:index+BATCH_SIZE]
                col += 1
            # continue for each of the 8 columns (8x10 = 80 columns)
        # set the new batch
        batchResponse[:,:,fpga] = fpgaBatch[:,:]
    return batchResponse

In [7]:
batched1 = group_frequencies(freqs1)
batched2 = group_frequencies(freqs2)
batched3 = group_frequencies(freqs3)
batched4 = group_frequencies(freqs4)

In [8]:
batched1[:,0,0] # first batch, first fpga

array([ 71870., 132424., 128502.,  87462.,  73009., 107253.,  62552.,
       185180.,  92504., 122187.,  83289., 131451., 123793., 607617.,
       505165., 101321.])

### Lehmer-Gray Encoding
Sorting the frequencies using Lehmer encoding.  
For each frequency $ f_i $, we generate $ s_i = |\{f_j | f_j < f_i \wedge j > i \}| $. The nice property is that if a single frequency swap occurs, the $ s_i $ coefficient only changes +-1.
Then these numbers are Gray-encoded, in order that for each +-1 in the $ s_i $ we have just one bit of change

In [9]:
def lehmer_encode(batch):
    n = len(batch)
    lehmanEncoded = np.zeros(n-1, dtype=np.int)
    for i in range(n-2, -1, -1):
        lehmanEncoded[i] = np.sum(batch[i+1:n] < batch[i])
    return lehmanEncoded

def gray_encode(s):
    # Gray encode each coefficient
    return [s[i] ^ (s[i] >> 1) for i in range(0, len(s))]

def num_to_binary(n, n_bits):
    out = np.zeros(n_bits, dtype=np.int)
    i = n_bits-1
    while(n > 0):
        if n >= pow(2,i):
            n = n - pow(2,i)
            out[i] = 1
        i = i - 1
    return out

def compress_encode(g, batch_len):
    if batch_len == 16:
        # encode gray values
        gray = np.zeros(7,dtype=np.int)
        gray[6] = g[14] + pow(2,1)*g[13] + pow(2,3)*g[12] + pow(2,5)*g[11]
        gray[5] = g[10] + pow(2,3)*g[9] + pow(2,6) * (3 & g[8])
        gray[4] = (4 & g[8])/pow(2,2) + 2*g[7] + pow(2,5) * (7 & g[6])
        gray[3] = (8 & g[6])/pow(2,3) + 2*g[5] + pow(2,5) * (7 & g[4])
        gray[2] = (8 & g[4])/pow(2,3) + 2*g[3] + pow(2,5) * (7 & g[2])
        gray[1] = (8 & g[2])/pow(2,3) + 2*g[1] + pow(2,5) * (7 & g[0])
        gray[0] = (1 & g[0])
        # convert to binary
        tmp = np.zeros((7,8))
        for i in range(0,7):
            tmp[i,:] = num_to_binary(gray[i], 8)
        # concatenate in a single array
        return np.concatenate([[tmp[0,0]], tmp[1,:], tmp[2,:], tmp[3,:], tmp[4,:], tmp[5,:], tmp[6,:]], axis=0)

def lehmer_gray_encode_batches(batched):
    BATCH_SIZE = batched.shape[0]   #16
    NUM_BATCHES = batched.shape[1]  #80
    NUM_FPGA = batched.shape[2]     #100
    BITS_PER_BATCH = 49
    out = np.zeros((BITS_PER_BATCH, NUM_BATCHES, NUM_FPGA), dtype=np.int)
    for curr_fpga in range(0,NUM_FPGA):
        for curr_batch in range(0, NUM_BATCHES):
            le = lehmer_encode(batched[:,curr_batch,curr_fpga])
            ge = gray_encode(le)
            out[:,curr_batch, curr_fpga] = compress_encode(ge, BATCH_SIZE)
    return out


In [10]:
data = np.array([
    lehmer_gray_encode_batches(batched1),
    lehmer_gray_encode_batches(batched2),
    lehmer_gray_encode_batches(batched3),
    lehmer_gray_encode_batches(batched4)
], dtype=np.int)

In [11]:
data.shape

(4, 49, 80, 100)

### Analysis

#### Uniformity
The uniformity metric (also called randomness by Yu et al. in [53])
determines how uniform the proportion of 0′ 𝑠 and 1′ 𝑠 are in the PUF
response

In [15]:
def calc_uniformity(response):
    return np.sum([response[i] for i in range(0, len(response))]) / len(response)

def worst_uniformity(data):
    max = 0.5   # ideal value
    for puf_num in range(0, data.shape[3]):
        for exp_num in range(0, data.shape[0]):
            for batch_num in range(0, data.shape[2]):
                u = calc_uniformity(data[exp_num,:,batch_num,puf_num])
                if np.abs(u - 0.5) > np.abs(max - 0.5):
                    max = u
    return max

In [16]:
worst_uniformity(data)

0.20408163265306123

#### Reliability
It determines how efficiently a PUF can generate the same response
at different operating conditions (ambient temperatures or supply volt-
ages) over a period of time for a given challenge

In [20]:
def hamming_distance(arr1, arr2): 
    distance = 0
    L = len(arr1)
    for i in range(L):
        if arr1[i] != arr2[i]:
            distance += 1
    return distance

def hd_intra(puf_responses):
    # 4x49
    return np.sum([
        hamming_distance(puf_responses[0,:], puf_responses[exp_num,:]) / puf_responses.shape[1]
        for exp_num in range(1, puf_responses.shape[0])]) / puf_responses.shape[0]
    
def reliability(puf_responses):
    return 1 - hd_intra(puf_responses)

def stat_reliability(data):
    min = 1 # ideal value
    max = 0
    sum = 0
    for puf_num in range(0, data.shape[3]):
        for batch_num in range(0, data.shape[2]):
            r = reliability(data[:,:,batch_num,puf_num])
            if r < min:
                min = r
            if r > max:
                max = r
            sum += r
    return min, max, sum / (data.shape[3] * data.shape[2])

In [21]:
stat_reliability(data)

(0.8163265306122449, 1.0, 0.9410669642857231)

#### Uniqueness
It measures the variation of responses obtained from different chips
for the same set of challenges

In [20]:
def calc_uniqueness(data):
    num_bits = data.shape[0]
    num_devices = data.shape[1]
    return 2/(num_devices*(num_devices-1)) * np.sum([
        np.sum([
            hamming_distance(data[:,i], data[:,j])/num_bits
            for j in range(i, num_devices)])  
        for i in range(0, num_devices-2)])

def worst_uniqueness(data):
    n_esperiments = data.shape[0]
    max = 0.5   # ideal value
    for exp_num in range(0, n_esperiments):
        for batch_num in range(0, data.shape[2]):
            u = calc_uniqueness(data[exp_num,:,batch_num,:])
            if np.abs(u - 0.5) > np.abs(max - 0.5):
                max = u
    return max

In [21]:
worst_uniqueness(data)

0.46561945990517417

### Some samples

In [26]:
def bit_array_to_string(arr):
    response_char = ''
    for i in range(0, len(arr)):
        response_char = response_char + ('1' if arr[i] else '0')
    return response_char  

def print_ids_per_fpga(data):
    num_batches = data.shape[1] #80
    out = []
    for curr_batch in range(0, num_batches):
        out.append(bit_array_to_string(data[:,curr_batch]))
    return out

In [28]:
# Let's print all the first 10 ids of the first board at 20°C
print_ids_per_fpga(data[0,:,:,0])[0:10]

['1101111000010010100110100010100000001101011110110',
 '0110000000001111001011011101101011101110011010011',
 '1000101001010010100110111101101001111110111101100',
 '1001101100001010001110100010001011110001111001010',
 '0001110000110010100010100001001011101001001010010',
 '0000000100000001100010101011001100100001011001011',
 '1111001100010010101100100010000100111011111010010',
 '0101010000110010100110001001100111001010110000110',
 '1001111101001011100011011000100010101111101001010',
 '1010011100001110101000111011101011100000011111100']